In [ ]:
import pandas as pd
import numpy as np
import pickle
import os
import onnxruntime as ort
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split

# Add project root to path to allow absolute imports
import sys

project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    from backend.ml.rto_env import CompressorEnv

# --- 1. Load Artifacts and Data ---
print("Loading data and artifacts...")
DATASET_PATH = "../datasets/MASTER_DATASET.csv"
MODEL_PATH = "../artifacts/best_ppo_actor.onnx"
SCALER_PATH = "../artifacts/rto_scaler.pkl"

df = pd.read_csv(DATASET_PATH)
session = ort.InferenceSession(MODEL_PATH)
input_name = session.get_inputs()[0].name
with open(SCALER_PATH, "rb") as f:
    scaler = pickle.load(f)
print("✅ Artifacts loaded.")


# --- 2. Prepare Test Environment ---
print("Preparing test environment...")
# Get the same state features used during training
temp_env = CompressorEnv(df.head(1))
state_features = temp_env.state_features
del temp_env

# Use the same train/test split as in training to get the test set
df_train, df_test = train_test_split(df, test_size=0.2, shuffle=False)

# Create the evaluation environment, using the scaler and dynamics from the training set
train_env_for_dynamics = CompressorEnv(df_train, scaler=scaler)
eval_env = CompressorEnv(
    df_test, scaler=scaler, dynamics_models=train_env_for_dynamics.dynamics_models
)
print("✅ Test environment ready.")


# --- 3. Run Backtesting Simulation ---
print("Running backtesting simulation...")
actual_efficiencies = []
rto_controlled_efficiencies = []
actions_taken = []

state, _ = eval_env.reset()
done = False
while not done:
    # Get the actual efficiency from the historical data
    actual_efficiencies.append(eval_env.df.loc[eval_env.current_step, "Efficiency"])

    # Get the model's suggested action
    state_tensor = state.reshape(1, -1).astype(np.float32)
    action = session.run(None, {input_name: state_tensor})[0][0][0]
    actions_taken.append(action)

    # Simulate what would have happened if we took this action
    next_state, reward, done, _, info = eval_env.step(np.array([action]))
    rto_controlled_efficiencies.append(info["efficiency"])

    state = next_state
print("✅ Backtesting complete.")


# --- 4. Visualize and Compare Results ---
print("Generating comparison plot...")
avg_actual = np.mean(actual_efficiencies)
avg_rto = np.mean(rto_controlled_efficiencies)
improvement = ((avg_rto - avg_actual) / avg_actual) * 100

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        y=actual_efficiencies,
        mode="lines",
        name=f"Actual Efficiency (Avg: {avg_actual:.4f})",
    )
)
fig.add_trace(
    go.Scatter(
        y=rto_controlled_efficiencies,
        mode="lines",
        name=f"RTO-Controlled Efficiency (Avg: {avg_rto:.4f})",
    )
)

fig.update_layout(
    title=f"RTO Backtesting Results: Potential Improvement of {improvement:.2f}%",
    xaxis_title="Time Steps (in test data)",
    yaxis_title="Compressor Efficiency",
)
fig.show()

Loading data and artifacts...
✅ Artifacts loaded.
Preparing test environment...
✅ Test environment ready.
Running backtesting simulation...
✅ Backtesting complete.
Generating comparison plot...
